In [570]:
%%capture
%pip install pandas_read_xml
%pip install --upgrade pip
!pip3 install upgrade-pip

In [571]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import numpy as np
#import sys
#print(sys.version)

In [572]:
def getStats(df,columnIndex):
    
    meanVar=round(df.iloc[:,columnIndex].mean(),3)
    statList=[meanVar,
              round(df.iloc[:,columnIndex].max(),3),
              round(df.iloc[:,columnIndex].min(),3),
              round(df.iloc[:,columnIndex].median(),3),
              round(df.iloc[:,columnIndex].quantile(0.25),3),
              round(df.iloc[:,columnIndex].quantile(0.75),3),
              round(df.iloc[:,columnIndex].std()/abs(meanVar),3)
             ]                   
    
    return statList

In [573]:
def getRates1(List,divisor):
    
    Rates=[]

    for item in List:
        Rates.append(round(item/divisor,3))
    
    return Rates

In [574]:
def getRates2(List,Dict,divisor):
    
    Rates=[]

    for item in List:
        Rates.append(round(sum([Dict[i] for i in item])/divisor,3))       
    
    return Rates

In [575]:
def getCharRates(df,colIndex,charList,sumLenWord):
    
    charRates=[]
    
    for char in charList:    
        charRates.append(round(df.iloc[:,colIndex].str.count(char).sum()/sumLenWord,3))
    
    return charRates

In [576]:
def getAuthorName(poems_path):
    
    try:
    
        df0 = pd.read_xml(poems_path, xpath="//ns:titleStmt/ns:author/ns:persName", namespaces={"ns":"http://www.tei-c.org/ns/1.0"})

        Author=df0.iloc[0, 0] + ' ' + df0.iloc[0, 1]

        AuthorDict={
            'Author':Author
        }

        return AuthorDict
    
    except Exception as e:
        AuthorDict={
            'Author':np.nan
        }
        return AuthorDict

In [577]:
def getDiv(poems_path):
    
    try:

        divDict={}

        divKeyList=['Title','numWordsTitle','numStanza','numLine','numWord','numSyll','rateShortSyll']

        df1 = pd.read_xml(poems_path, xpath="//div")

        special_characters=['!',':',';','"','-','.','?',',','(',')','[',']','»','«','ʼ']

        Title=df1.iloc[0, 9]
        numWordsTitle=len(Title.replace(f"[{''.join(special_characters)}]",'').split())
        numStanza=df1.iloc[0, 1]
        numLine=df1.iloc[0, 2]
        numWord=df1.iloc[0, 3]
        numSyll=df1.iloc[0, 4]
        rateShortSyll=round(df1.iloc[0, 5] / numSyll,3)

        divValueList=[Title,numWordsTitle,numStanza,numLine,numWord,numSyll,rateShortSyll]
        divDict.update(dict(zip(divKeyList,divValueList)))

        return divDict

    except Exception as e:
        divDict=dict.fromkeys(divKeyList,np.nan)
        return divDict

In [578]:
def getLg(poems_path):
    
    lgKeyList1=['meanNL','maxNL','minNL','medNL','q1NL','q3NL','vNL']
    lgKeyList2=['meanNW','maxNW','minNW','medNW','q1NW','q3NW','vNW']
    lgKeyList3=['meanNS','maxNS','minNS','medNS','q1NS','q3NS','vNS']
    lgKeyList4=['meanNshortS','maxNshortS','minNshortS','medNshortS','q1NshortS','q3NshortS','vNshortS']
    lgKeyList5=['meanNlongS','maxNlongS','minNlongS','medNlongS','q1NlongS','q3NlongS','vNlongS']
    lgKeyList6=['abcb','aba','abcdb','abca','abb','aaaa']
    
    try:

        lgDict={}

        df2 = pd.read_xml(poems_path, xpath="//lg")    

        lgDict.update(dict(zip(lgKeyList1,getStats(df2,1))))
        lgDict.update(dict(zip(lgKeyList2,getStats(df2,2))))
        lgDict.update(dict(zip(lgKeyList3,getStats(df2,3))))
        lgDict.update(dict(zip(lgKeyList4,getStats(df2,4))))
        lgDict.update(dict(zip(lgKeyList5,getStats(df2,5))))

        rhymeFormCountList=df2[df2.iloc[:,6].isin(lgKeyList6)].iloc[:,6].value_counts().tolist()
        lgDict.update(dict(zip(lgKeyList6,rhymeFormCountList)))

        return lgDict

    except Exception as e:
        lgDict=dict.fromkeys(lgKeyList1+lgKeyList2+lgKeyList3+lgKeyList4+lgKeyList5+lgKeyList6,np.nan)
        return lgDict

In [579]:
def getL(poems_path):
    
    lKeyList1=['LmeanNW','LmaxNW','LminNW','LmedNW','Lq1NW','Lq3NW','LvNW']
    lKeyList2=['LmeanNS','LmaxNS','LminNS','LmedNS','Lq1NS','Lq3NS','LvNS']
    lKeyList3=['LmeanNshortS','LmaxNshortS','LminNshortS','LmedNshortS','Lq1NshortS','Lq3NshortS','LvNshortS']
    lKeyList4=['LmeanNlongS','LmaxNlongS','LminNlongS','LmedNlongS','Lq1NlongS','Lq3NlongS','LvNlongS']
    
    try:
    
        lDict={}

        df3 = pd.read_xml(poems_path, xpath="//l")
        
        lDict.update(dict(zip(lKeyList1,getStats(df3,1))))
        lDict.update(dict(zip(lKeyList2,getStats(df3,2))))
        lDict.update(dict(zip(lKeyList3,getStats(df3,3))))
        lDict.update(dict(zip(lKeyList4,getStats(df3,4))))

        return lDict

    except Exception as e:
        lDict=dict.fromkeys(lKeyList1+lKeyList2+lKeyList3+lKeyList4,np.nan)
        return lDict

In [580]:
def getW(poems_path):
    
    wKeyList1=['PROPN','NOUN','ADJ','NUM','PRON','VERB','ADV','CONJ','SCONJ','DET','ADP','INTJ','PART']
    wKeyList2=['ratePROPN_NOUN','rateADJ','rateNOUNs','rateVERB','rateADV']
    wKeyList3=['ratePhonTlow','ratePhonThigh','ratePhonTmixed']
    wKeyList4=['meanLenWord','maxLenWord','minLenWord','medLenWord','q1LenWord','q3LenWord','vLenWord']
    wKeyList5=['sumLenWord']
    wKeyList6=['rateConsonant','rateShortFFV','rateLongFFV','rateShortBFV','rateLongBFV']    
    
    try:

        wDict={}

        df1 = pd.read_xml(poems_path, xpath="//div")
        numWord=df1.iloc[0, 3]

        df4 = pd.read_xml(poems_path, xpath="//w")
        
        nounsCountList=df4[df4.iloc[:,2].isin(wKeyList1)].iloc[:,2].value_counts().tolist()
        wDict.update(dict(zip(wKeyList1,nounsCountList)))

        nounsList=[['PROPN','NOUN'],['ADJ'],['PROPN','NOUN','ADJ','NUM','PRON'],['VERB'],['ADV']]
        wDict.update(dict(zip(wKeyList2,getRates2(nounsList,wDict,numWord))))

        vowelsList=['low','high','mixed']
        vowelsCountList=df4[df4.iloc[:,5].isin(vowelsList)].iloc[:,5].value_counts().tolist()
        wDict.update(dict(zip(wKeyList3,getRates1(vowelsCountList,numWord))))

        df4['lenWord']=df4.iloc[:, 6].str.len()
        wDict.update(dict(zip(wKeyList4,getStats(df4,8))))

        wDict[wKeyList5[0]]=df4.iloc[:,8].sum()
        charList=['c','f','F','b','B']
        wDict.update(dict(zip(wKeyList6,getCharRates(df4,6,charList,wDict['sumLenWord']))))

        return wDict
    
    except Exception as e:
        wDict=dict.fromkeys(wKeyList1+wKeyList2+wKeyList3+wKeyList4+wKeyList5+wKeyList6,np.nan)
        return wDict

In [581]:
def getPC(poems_path):
    
    try:
        
        df5 = pd.read_xml(poems_path, xpath="//pc")

        PUNCT=(df5.iloc[:, 1]=='PUNCT').sum()

        PUNCTdict={'PUNCT':PUNCT}

        return PUNCTdict
    
    except Exception as e:
        PUNCTdict={'PUNCT':np.nan}
        return PUNCTdict

In [582]:
def getRhymePair(poems_path):
    
    try:

        df6 = pd.read_xml(poems_path, xpath="//rhymePair")

        numRhymePairs=df6.iloc[:, 2].count()

        rhymePairDict={'numRhymePairs':numRhymePairs}

        return rhymePairDict
    
    except Exception as e:
        rhymePairDict={'numRhymePairs':np.nan}
        return rhymePairDict

In [583]:
def getAlliteration(poems_path):
    
    allKeyList1=['numAll','rateClearAll']
    allKeyList2=['meanAll','maxAll','minAll','medAll','q1All','q3All','vAll']
    
    try:
        
        AllDict={}

        df7 = pd.read_xml(poems_path, xpath="//alliteration")

        AllDict[allKeyList1[0]]=df7.iloc[:, 1].count()

        df7['lenAll']=df7.iloc[:, 1].apply(len)
        AllDict[allKeyList1[1]]=round((df7.iloc[:, 1].str.count('a')==df7.iloc[:, 6]).sum()/AllDict['numAll'],3)

        AllDict.update(dict(zip(allKeyList2,getStats(df7,6))))

        return AllDict
    
    except Exception as e:
        AllDict=dict.fromkeys(allKeyList1+allKeyList2,np.nan)
        return AllDict

In [584]:
# Kiindulási mappa, amiben a további almappák vannak
poemsPath="D:\Iskolai\Tananyagok\Szakdolgozat\Bsc\Versek"

rawIndex=0

df=pd.DataFrame()

# os.walk használata az összes almappa bejárásához
for rootFolder, Folder, Files in os.walk(poemsPath):
    for File in Files:
        if File.endswith('.xml'):  # Csak az XML kiterjesztésű fájlokat vesszük figyelembe
            completePath = os.path.join(rootFolder, File)

            # XML fájl beolvasása
            try:
                tree = ET.parse(completePath)
                root = tree.getroot()

                # Itt dolgozhatsz az XML tartalommal
                # Például: print(root.tag)

                functionList=[getAuthorName,getDiv,getLg,getL,getW,getPC,getRhymePair,getAlliteration]
                
                mainDict={}
                
                for function in functionList:
                    mainDict.update(function(completePath))    
                
                if rawIndex==0:                        

                    column_names=list(mainDict.keys())            

                    df = pd.DataFrame(columns=column_names)                   

                for item in mainDict:    

                    df.loc[rawIndex, item] = mainDict[item]

                rawIndex+=1

            except ET.ParseError:
                print(f'Hiba a(z) {File} XML fájl olvasásakor.')

#print(mainDictionary)

In [603]:
df
# reszlet=df.loc[0:59,'abcb':'abcdb']
# reszlet

,Author,Title,numWordsTitle,numStanza,numLine,numWord,numSyll,rateShortSyll,meanNL,maxNL,...,maxAll,minAll,medAll,q1All,q3All,vAll,INTJ,aba,abcdb,PART
0,Ady Endre,GÓG ÉS MAGÓG FIA VAGYOK ÉN...,6,4,16,82,157,0.503,4.0,4,...,3,2,3.0,2.0,3.0,0.211,NaN,NaN,NaN,NaN
1,Ady Endre,A MI GYERMEKÜNK,3,5,25,108,210,0.448,5.0,5,...,4,2,2.5,2.0,3.0,0.306,NaN,NaN,NaN,NaN
2,Ady Endre,A VÁR FEHÉR ASSZONYA,4,4,16,77,136,0.529,4.0,4,...,4,3,3.5,3.25,3.75,0.202,NaN,NaN,NaN,NaN
3,Ady Endre,MERT ENGEM SZERETSZ,3,1,15,43,75,0.453,15.0,15,...,3,3,3.0,3.0,3.0,0.0,NaN,NaN,NaN,NaN
4,Ady Endre,A KÖNNYEK ASSZONYA [1],4,3,24,97,180,0.45,8.0,8,...,4,2,2.0,2.0,2.0,0.285,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,Ady Endre,UJJAK A SZAJNÁBAN,3,3,15,62,132,0.455,5.0,5,...,3,2,2.5,2.25,2.75,0.283,NaN,NaN,NaN,NaN
1112,Ady Endre,"ÁLDOTT, FALUSI KÖD",3,5,20,92,180,0.433,4.0,4,...,6,2,3.0,3.0,3.0,0.381,NaN,NaN,NaN,NaN
1113,Ady Endre,AZ ISTEN-KERESŐ LÁRMA,3,2,12,44,86,0.477,6.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1114,Ady Endre,KI LÁTOTT ENGEM?,3,11,22,118,242,0.471,2.0,2,...,3,2,3.0,2.0,3.0,0.206,NaN,NaN,NaN,NaN
